# SPCS MySQL Database Connectivity Test

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

This notebook provides modular components for testing MySQL connectivity from Snowflake Container Services (SPCS):

## Quick Start Options:
1. **Basic Test**: Configure parameters and run network/auth tests
2. **Full Setup**: Use cells to set up PyPI access and MySQL EAI as needed
3. **Custom Setup**: Pick and choose individual components as needed

## Components Available:
- **Configuration**: Set your MySQL connection parameters
- **PyPI Setup**: Optional - enables installing mysql-connector-python driver from PyPI
- **Network Test**: Tests basic network connectivity to MySQL host
- **Authentication Test**: Tests MySQL authentication and basic queries using mysql-connector-python
- **MySQL EAI**: Optional - configures External Access Integration for MySQL


**Note: You must restart the Session after making EAI changes for them to take effect**

In [ ]:
# MySQL Connectivity Test Configuration
# Update these parameters with your actual MySQL connection details

# User Configuration - UPDATE THIS
MYSQL_HOST = "database-sk-1-instance-1.abcdef123.us-east-2.rds.amazonaws.com"
MYSQL_PORT = 3306
MYSQL_DATABASE = "database"
MYSQL_USER = "admin"
MYSQL_PASSWORD = "password"

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# This role will be used by Openflow to run Connectors and perform your Data Engineering tasks
OPENFLOW_RUNTIME_ROLE = "OPENFLOWRUNTIMEROLE"

print(f"Configuration:")
print(f"MySQL Host: {MYSQL_HOST}")
print(f"MySQL Port: {MYSQL_PORT}")
print(f"MySQL Database: {MYSQL_DATABASE}")
print(f"MySQL User: {MYSQL_USER}")
print("\nReady to test connectivity...")

## 1. PyPI Setup (Optional)

Run these cells if you need to install the mysql-connector-python MySQL driver from PyPI. This creates the necessary network rules and External Access Integration for PyPI access.

**Skip this section if you already have mysql-connector-python installed or have PyPI access configured.**


In [ ]:
-- Create Network Rule and External Access Integration for PyPI
-- Run this cell to enable installing Python packages from PyPI

USE ROLE {{IMPLEMENTATION_ROLE}};

CREATE OR REPLACE NETWORK RULE pypi_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('pypi.org', 'pypi.python.org', 'pythonhosted.org', 'files.pythonhosted.org');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION pypi_access_integration
  ALLOWED_NETWORK_RULES = (pypi_network_rule)
  ENABLED = true
  COMMENT = 'External Access Integration for PyPI package installation';

-- Grant usage on the integration
GRANT USAGE ON INTEGRATION pypi_access_integration TO ROLE ACCOUNTADMIN;

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'pypi_access_integration';


In [ ]:
-- Apply PyPI integration to this notebook
-- Run this after creating the PyPI integration above

ALTER NOTEBOOK EAI_MYSQL
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('pypi_access_integration');


**NOTE: You will need to restart the Notebook session after applying an EAI**

In [ ]:
# Install MySQL driver (mysql-connector-python)
# Make sure PyPI access is configured first if you get connection errors
# You can run this cell twice; the first to install the driver, the second to confirm it is imported

try:
    import mysql.connector
    print("✅ mysql-connector-python already available")
except ImportError:
    print("📦 Installing mysql-connector-python...")
    %pip install mysql-connector-python
    print("✅ mysql-connector-python installed")

## 2. Connectivity Tests

These cells test connectivity to your MySQL database. Run them in order to diagnose any connection issues.

**Note: If you need to install mysql-connector-python, make sure to run the PyPI setup section first and restart your session.**


In [ ]:
# Network Connectivity Test
# Tests basic network connectivity to MySQL host and port

import socket

def test_network_connectivity(host, port, description):
    """Test network connectivity to MySQL host and port"""
    try:
        print(f"🔍 Testing {description}: {host}:{port}")

        # Test network connectivity
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(10)
        result = sock.connect_ex((host, port))
        sock.close()

        if result == 0:
            print(f"  ✅ Network connection successful")
            return True
        else:
            print(f"  ❌ Network connection failed (error code: {result})")
            print(f"  💡 This may indicate the need for a MySQL EAI")
            return False

    except Exception as e:
        print(f"  ❌ Network Error: {e}")
        return False

# Run network connectivity test
print("=" * 50)
print("NETWORK CONNECTIVITY TEST")
print("=" * 50)

network_result = test_network_connectivity(MYSQL_HOST, MYSQL_PORT, "MySQL Network Connectivity")

if network_result:
    print(f"\n✅ Network connectivity PASSED - MySQL host is reachable")
    print("You can proceed to test MySQL authentication.")
else:
    print(f"\n❌ Network connectivity FAILED")
    print("You need to configure a MySQL External Access Integration (EAI).")
    print("See the MySQL EAI Setup section below.")

In [ ]:
# MySQL Authentication Test
# Tests MySQL authentication and basic database operations
# Run this after confirming network connectivity works

def test_mysql_authentication():
    """Test MySQL authentication and basic query"""
    try:
        print(f"🔍 Testing MySQL Authentication and Basic Query")

        # Import mysql.connector library
        try:
            import mysql.connector
            print("  📦 Using mysql-connector-python library")
        except ImportError:
            print("  ❌ mysql-connector-python library not available")
            print("  💡 Run the PyPI setup section first to install mysql-connector-python")
            return False

        # Create database connection
        conn = mysql.connector.connect(
            host=MYSQL_HOST,
            port=MYSQL_PORT,
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD
        )

        print(f"  ✅ Authentication successful")

        # Test basic query
        cursor = conn.cursor()
        cursor.execute("SELECT VERSION();")
        version = cursor.fetchone()[0]
        print(f"  ✅ Database query successful")
        print(f"  📊 MySQL Version: {version[:50]}...")

        # List tables
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        print(f"  📋 Found {len(tables)} tables in database")
        # Show first 3 tables
        for i, table in enumerate(tables[:3]):
            print(f"     - {table[0]}")
        if len(tables) > 3:
            print(f"     ... and {len(tables) - 3} more tables")

        cursor.close()
        conn.close()
        return True

    except Exception as e:
        print(f"  ❌ Database Error: {e}")
        print(f"  💡 Check your credentials or network connectivity")
        return False

# Run MySQL authentication test
print("=" * 50)
print("MYSQL AUTHENTICATION TEST")
print("=" * 50)

auth_result = test_mysql_authentication()

if auth_result:
    print(f"\n✅ MySQL authentication PASSED")
    print("Your SPCS environment can successfully connect to MySQL!")
    print("You can proceed with MySQL integration.")
else:
    print(f"\n❌ MySQL authentication FAILED")
    print("Check your credentials and ensure network connectivity is working.")
    print("You may also need to configure MySQL EAI (see section below).")


## 2a. MySQL Replication Configuration Validation (Optional)

This section validates that your MySQL instance is properly configured for Snowflake Openflow connector integration. It checks the key prerequisites outlined in the [Snowflake documentation](https://docs.snowflake.com/en/user-guide/data-integration/openflow/connectors/mysql/setup#prerequisites).

**Run this cell if you plan to use Snowflake Openflow connector for MySQL data integration.**

**Skip this section if you only need basic MySQL connectivity testing.**

The validation checks:
- Binary logging configuration (log_bin, binlog_format, binlog_row_metadata, etc.)
- Buffer sizes and retention settings
- MySQL version compatibility (8+)
- Replica-specific settings (if applicable)


In [ ]:
# MySQL Replication Configuration Validation (Optional)
# Validates MySQL settings required for Snowflake Openflow connector
# Run this after successful authentication to check replication prerequisites

def validate_mysql_replication_config():
    """Validate MySQL configuration for Snowflake Openflow connector"""
    try:
        print(f"🔍 Validating MySQL Replication Configuration")

        # Import mysql.connector library
        try:
            import mysql.connector
            print("  📦 Using mysql-connector-python library")
        except ImportError:
            print("  ❌ mysql-connector-python library not available")
            return False

        # Create database connection
        conn = mysql.connector.connect(
            host=MYSQL_HOST,
            port=MYSQL_PORT,
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD
        )

        cursor = conn.cursor()

        # Check required MySQL system variables
        required_settings = {
            'log_bin': {'expected': 'ON', 'description': 'Binary logging enabled'},
            'binlog_format': {'expected': 'ROW', 'description': 'Row-based replication format'},
            'binlog_row_metadata': {'expected': 'FULL', 'description': 'Full row metadata in binlog'},
            'binlog_row_image': {'expected': 'FULL', 'description': 'Full row image in binlog'},
            'binlog_row_value_options': {'expected': '', 'description': 'Empty row value options'},
            'binlog_expire_logs_seconds': {'min_value': 3600, 'description': 'Binlog retention (min 1 hour recommended)'},
            'sort_buffer_size': {'min_value': 4194304, 'description': 'Sort buffer size (min 4MB recommended)'}
        }

        print("  📋 Checking MySQL replication configuration...")
        all_valid = True

        for setting, config in required_settings.items():
            try:
                cursor.execute(f"SHOW VARIABLES LIKE '{setting}';")
                result = cursor.fetchone()

                if result:
                    current_value = result[1]

                    if 'expected' in config:
                        # Exact match validation
                        if current_value.upper() == config['expected'].upper():
                            print(f"    ✅ {setting}: {current_value} - {config['description']}")
                        else:
                            print(f"    ❌ {setting}: {current_value} (expected: {config['expected']}) - {config['description']}")
                            all_valid = False

                    elif 'min_value' in config:
                        # Minimum value validation
                        try:
                            current_int = int(current_value)
                            if current_int >= config['min_value']:
                                print(f"    ✅ {setting}: {current_value} - {config['description']}")
                            else:
                                print(f"    ⚠️  {setting}: {current_value} (recommended min: {config['min_value']}) - {config['description']}")
                                # Don't mark as invalid for recommendations
                        except ValueError:
                            print(f"    ❓ {setting}: {current_value} (unable to validate) - {config['description']}")

                else:
                    print(f"    ❓ {setting}: Not found - {config['description']}")

            except Exception as e:
                print(f"    ❌ Error checking {setting}: {e}")
                all_valid = False

        # Check if this is a replica and validate replica settings
        try:
            cursor.execute("SHOW SLAVE STATUS;")
            replica_status = cursor.fetchone()

            if replica_status:
                print("  📋 Detected MySQL replica - checking replica-specific settings...")
                cursor.execute("SHOW VARIABLES LIKE 'log_replica_updates';")
                result = cursor.fetchone()

                if result:
                    if result[1].upper() == 'ON':
                        print(f"    ✅ log_replica_updates: {result[1]} - Replica logging enabled")
                    else:
                        print(f"    ❌ log_replica_updates: {result[1]} (expected: ON) - Replica logging disabled")
                        all_valid = False
                else:
                    print(f"    ❓ log_replica_updates: Not found")
            else:
                print("  📋 This is a primary MySQL instance (not a replica)")

        except Exception as e:
            print(f"  📋 Could not check replica status: {e}")

        # Check MySQL version compatibility
        cursor.execute("SELECT VERSION();")
        version = cursor.fetchone()[0]
        version_parts = version.split('.')
        major_version = int(version_parts[0])

        if major_version >= 8:
            print(f"  ✅ MySQL Version: {version} - Compatible (MySQL 8+ required)")
        else:
            print(f"  ❌ MySQL Version: {version} - Not compatible (MySQL 8+ required)")
            all_valid = False

        cursor.close()
        conn.close()

        return all_valid

    except Exception as e:
        print(f"  ❌ Configuration check failed: {e}")
        return False

# Run MySQL replication configuration validation
print("=" * 60)
print("MYSQL REPLICATION CONFIGURATION VALIDATION")
print("=" * 60)

config_valid = validate_mysql_replication_config()

if config_valid:
    print(f"\n✅ MySQL replication configuration is VALID")
    print("Your MySQL instance appears to be properly configured for Snowflake Openflow connector.")
    print("You can proceed with setting up the Openflow connector per the documentation.")
else:
    print(f"\n⚠️  MySQL replication configuration has ISSUES")
    print("Some settings may need to be adjusted for optimal Snowflake integration.")
    print("Review the settings above and consult your MySQL administrator.")
    print("\nFor more details, see:")
    print("https://docs.snowflake.com/en/user-guide/data-integration/openflow/connectors/mysql/setup#prerequisites")


## 3. MySQL EAI Setup (Optional)

Run these cells if connectivity tests failed. This creates the necessary network rules and External Access Integration specifically for MySQL database access.

**Skip this section if your connectivity tests passed.**

In [ ]:
-- Create Network Rule for MySQL connectivity
-- This allows egress traffic to your specific MySQL host and port

USE ROLE {{IMPLEMENTATION_ROLE}};

-- Network rule for your MySQL database server
-- Uses the variables set in the configuration cell above
CREATE OR REPLACE NETWORK RULE MYSQL_DB_RULE
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('{{MYSQL_HOST}}:{{MYSQL_PORT}}');

-- Alternative network rule for general AWS RDS MySQL access (if needed)
-- Uncomment and modify if you need broader AWS RDS access
-- CREATE OR REPLACE NETWORK RULE mysql_aws_rds_rule
--   MODE = EGRESS
--   TYPE = HOST_PORT
--   VALUE_LIST = ('*.rds.amazonaws.com:3306');

-- Show the created network rule
DESCRIBE NETWORK RULE MYSQL_DB_RULE;

In [ ]:
-- Create External Access Integration for MySQL
-- This combines all the network rules into a single integration

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION MYSQL_EAI
  ALLOWED_NETWORK_RULES = (
    MYSQL_DB_RULE
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for MySQL database connectivity';

-- Grant usage on the integration to your role
GRANT USAGE ON INTEGRATION MYSQL_EAI TO ROLE {{IMPLEMENTATION_ROLE}};
GRANT USAGE ON INTEGRATION MYSQL_EAI TO ROLE {{OPENFLOW_RUNTIME_ROLE}};

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'MYSQL_EAI';

In [ ]:
-- Apply MySQL EAI to this notebook
-- This enables the notebook to access your MySQL database
-- Include pypi_access_integration if you created it earlier

ALTER NOTEBOOK EAI_MYSQL
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('MYSQL_EAI', 'pypi_access_integration');

-- Alternative: If you only need MySQL access (no PyPI), use:
-- ALTER NOTEBOOK MYSQL
--   SET EXTERNAL_ACCESS_INTEGRATIONS = ('MYSQL_EAI');